# <span style="color: var(--vscode-foreground);"><b>1. Tổng quan dữ liệu</b></span>

In [1]:
SELECT TOP 10 *
FROM user_behavior_dataset

(10 rows affected)

Total execution time: 00:00:00.077

User_ID,Device_Model,Operating_System,App_Usage_Time_min_day,Screen_On_Time_hours_day,Battery_Drain_mAh_day,Number_of_Apps_Installed,Data_Usage_MB_day,Age,Gender,User_Behavior_Class
1,Google Pixel 5,Android,393,6.400000095367432,1872,67,1122,40,Male,4
2,OnePlus 9,Android,268,4.699999809265137,1331,42,944,47,Female,3
3,Xiaomi Mi 11,Android,154,4,761,32,322,42,Male,2
4,Google Pixel 5,Android,239,4.800000190734863,1676,56,871,20,Male,3
5,iPhone 12,iOS,187,4.300000190734863,1367,58,988,31,Female,3
6,Google Pixel 5,Android,99,2,940,35,564,31,Male,2
7,Samsung Galaxy S21,Android,350,7.300000190734863,1802,66,1054,21,Female,4
8,OnePlus 9,Android,543,11.399999618530273,2956,82,1702,31,Male,5
9,Samsung Galaxy S21,Android,340,7.699999809265137,2138,75,1053,42,Female,4
10,iPhone 12,iOS,424,6.599999904632568,1957,75,1301,42,Male,4


Mô tả bộ dữ liệu: Bộ dữ liệu này gồm **700 bản ghi** (mỗi dòng đại diện cho một người dùng điện thoại di động), với **11 cột** mô tả hành vi và đặc điểm sử dụng thiết bị của họ.

- **User\_ID**: Mã định danh của người dùng
- **Device\_Model**: Tên thiết bị di động
- **Operating\_System**: Hệ điều hành
- **App\_Usage\_Time\_min\_day**: Tổng thời gian sử dụng ứng dụng trung bình mỗi ngày (phút)
- **Screen\_On\_Time\_hours\_day**: Thời gian bật màn hình trung bình mỗi ngày (giờ)
- **Battery\_Drain\_mAh\_day**: Lượng pin tiêu thụ trung bình mỗi ngày
- **Number\_of\_Apps\_Installed**: Số lượng ứng dụng được cài đặt
- **Data\_Usage\_MB\_day**: Lượng dữ liệu (MB) sử dụng trung bình mỗi ngày
- **Age**: Tuổi người dùng
- **Gender**: Giới tính
- **User\_Behavior\_Class**: Phân loại nhóm hành vi người dùng

## **1.1 Tổng số người dùng, thiết bị, hệ điều hành, giới tính và nhóm người dùng**

In [2]:
SELECT
   COUNT(*) AS total_user,
   COUNT(DISTINCT Device_Model) AS model_count,
   COUNT(DISTINCT Operating_System) AS os_count,
   COUNT(DISTINCT Gender) AS gender_count,
   COUNT(DISTINCT User_Behavior_Class) AS class_count
FROM user_behavior_dataset

(1 row affected)

Total execution time: 00:00:00.047

total_user,model_count,os_count,gender_count,class_count
700,5,2,2,5


## **1.2 Số người dùng theo hệ điều hành**

In [3]:
SELECT
   Operating_System,
   COUNT(*) AS total_user
FROM user_behavior_dataset
GROUP BY Operating_System
ORDER BY total_user DESC

(2 rows affected)

Total execution time: 00:00:00.038

Operating_System,total_user
Android,554
iOS,146


## **1.3 Số người dùng theo giới tính**

In [4]:
SELECT
   Gender,
   COUNT(*) AS total_user
FROM user_behavior_dataset
GROUP BY Gender
ORDER BY total_user DESC

(2 rows affected)

Total execution time: 00:00:00.024

Gender,total_user
Male,364
Female,336


## **1.4 Thống kê trung bình thời gian sử dụng ứng dụng và màn hình theo OS**

In [5]:
SELECT Operating_System,
       ROUND(AVG(App_Usage_Time_min_day), 2) AS avg_app_time,
       ROUND(AVG(Screen_On_Time_hours_day), 2) AS avg_screen_time
FROM user_behavior_dataset
GROUP BY Operating_System

(2 rows affected)

Total execution time: 00:00:00.027

Operating_System,avg_app_time,avg_screen_time
Android,268,5.23
iOS,282,5.43


## **1.5 Số người dùng theo thiết bị**

In [6]:
SELECT TOP 10
    Device_Model,
    COUNT(*) AS total_user,
    ROUND(AVG("App_Usage_Time_min_day"),2) AS avg_app_time
FROM user_behavior_dataset
GROUP BY Device_Model
ORDER BY total_user DESC

(5 rows affected)

Total execution time: 00:00:00.038

Device_Model,total_user,avg_app_time
Xiaomi Mi 11,146,268
iPhone 12,146,282
Google Pixel 5,142,267
Samsung Galaxy S21,133,266
OnePlus 9,133,270


# **2. Phân tích mức độ sử dụng**

## **2.1  Tương quan giữa app usage, pin và data**

In [7]:
SELECT
    ROUND(AVG(App_Usage_Time_min_day),2) AS avg_app_time,
    ROUND(AVG(Battery_Drain_mAh_day),2) AS avg_battery_drain,
    ROUND(AVG(Data_Usage_MB_day),2) AS avg_data_usage
FROM user_behavior_dataset

(1 row affected)

Total execution time: 00:00:00.018

avg_app_time,avg_battery_drain,avg_data_usage
271,1525,929


## **2.2 So sánh theo từng nhóm hành vi**

In [8]:
SELECT User_Behavior_Class,
       ROUND(AVG(App_Usage_Time_min_day),2) AS avg_app_time,
       ROUND(AVG(Battery_Drain_mAh_day),2) AS avg_battery_drain,
       ROUND(AVG(Data_Usage_MB_day),2) AS avg_data_usage,
       COUNT(*) AS total_user
FROM user_behavior_dataset
GROUP BY User_Behavior_Class
ORDER BY User_Behavior_Class


(5 rows affected)

Total execution time: 00:00:00.022

User_Behavior_Class,avg_app_time,avg_battery_drain,avg_data_usage,total_user
1,60,454,202,136
2,131,883,451,146
3,235,1515,822,143
4,395,2105,1232,139
5,541,2701,1974,136


\-\> nhóm 5 là nhóm sử dụng app nhiều nhất

# **3. Phân tích nhân khẩu học**

## **3.1  Ảnh hưởng của giới tính đến hành vi**

In [9]:
SELECT Gender,
       ROUND(AVG(App_Usage_Time_min_day),2) AS avg_app_time,
       ROUND(AVG(Number_of_Apps_Installed),2) AS avg_app,
       ROUND(AVG(Data_Usage_MB_day),2) AS avg_data_usage
FROM user_behavior_dataset
GROUP BY Gender

(2 rows affected)

Total execution time: 00:00:00.017

Gender,avg_app_time,avg_app,avg_data_usage
Female,271,50,914
Male,270,51,943


\-\> nam, nữ đều sử dụng app và có số lượng app tương đương nhau

## **3.2 Ảnh hưởng của độ tuổi**

In [10]:
SELECT
    CASE
        WHEN Age < 25 THEN 'Dưới 25'
        WHEN Age BETWEEN 25 AND 40 THEN '25-40'
        ELSE 'Trên 40'
    END AS age_group,
    ROUND(AVG(App_Usage_Time_min_day),2) AS avg_app_time,
    ROUND(AVG(Data_Usage_MB_day),2) AS avg_data_usage,
    ROUND(AVG(Battery_Drain_mAh_day),2) AS avg_battery_drain
FROM user_behavior_dataset
GROUP BY
    CASE
        WHEN Age < 25 THEN 'Dưới 25'
        WHEN Age BETWEEN 25 AND 40 THEN '25-40'
        ELSE 'Trên 40'
    END
ORDER BY avg_app_time,avg_data_usage,avg_battery_drain


(3 rows affected)

Total execution time: 00:00:00.024

age_group,avg_app_time,avg_data_usage,avg_battery_drain
25-40,259,897,1465
Trên 40,271,929,1527
Du?i 25,297,1012,1668


\-\> nhóm dưới 25 tuổi sử dụng app nhiều nhất, tiêu thụ data và pin nhiều nhất

# **4. Phân khúc người dùng**

Chia nhóm theo mức sử dụng

In [11]:
SELECT
    CASE
        WHEN App_Usage_Time_min_day > 300 THEN 'Heavy User'
        WHEN App_Usage_Time_min_day BETWEEN 150 AND 300 THEN 'Moderate User'
        ELSE 'Light User'
    END AS usage_roup,
    COUNT(*) AS total_user,
    ROUND(AVG(Battery_Drain_mAh_day),2) AS avg_battery_drain,
    ROUND(AVG(Data_Usage_MB_day),2) AS avg_data_usage,
    ROUND(AVG(Number_of_Apps_Installed),2) AS avg_app
FROM user_behavior_dataset
GROUP BY
    CASE
        WHEN App_Usage_Time_min_day > 300 THEN 'Heavy User'
        WHEN App_Usage_Time_min_day BETWEEN 150 AND 300 THEN 'Moderate User'
        ELSE 'Light User'
    END
ORDER BY total_user DESC

(3 rows affected)

Total execution time: 00:00:00.019

usage_roup,total_user,avg_battery_drain,avg_data_usage,avg_app
Heavy User,275,2400,1599,79
Light User,239,636,310,21
Moderate User,186,1373,735,45


# **5. So sánh theo hệ điều hành**

In [12]:
SELECT Operating_System,
       ROUND(AVG(App_Usage_Time_min_day),2) AS avg_app_time,
       ROUND(AVG(Battery_Drain_mAh_day),2) AS avg_battery_drain,
       ROUND(AVG(Data_Usage_MB_day),2) AS avg_data_usage,
       ROUND(AVG(Number_of_Apps_Installed),2) AS avg_app
FROM user_behavior_dataset
GROUP BY Operating_System

(2 rows affected)

Total execution time: 00:00:00.020

Operating_System,avg_app_time,avg_battery_drain,avg_data_usage,avg_app
Android,268,1508,920,50
iOS,282,1589,965,52


\-\> người dùng iOS có mức sử dụng cao hơn Android

# **6. Tổng kết hành vi theo lớp**

In [13]:
SELECT User_Behavior_Class,
       ROUND(AVG(Age),2) AS avg_age,
       ROUND(AVG(App_Usage_Time_min_day),2) AS avg_app_time,
       ROUND(AVG(Screen_On_Time_hours_day),2) AS avg_screen_time,
       ROUND(AVG(Battery_Drain_mAh_day),2) AS avg_battery_drain,
       ROUND(AVG(Data_Usage_MB_day),2) AS avg_data_usage,
       ROUND(AVG(Number_of_Apps_Installed),2) AS avg_app
FROM user_behavior_dataset
GROUP BY User_Behavior_Class
ORDER BY User_Behavior_Class

(5 rows affected)

Total execution time: 00:00:00.019

User_Behavior_Class,avg_age,avg_app_time,avg_screen_time,avg_battery_drain,avg_data_usage,avg_app
1,38,60,1.49,454,202,14
2,38,131,3.04,883,451,30
3,38,235,4.96,1515,822,50
4,38,395,6.91,2105,1232,69
5,38,541,10.11,2701,1974,89
